# Convolution

Load the modules required.

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

## First approach

Consider an image encoded as an $M \times N$ matrix $I$.  A filter is an $m \times n$ matrix $F$, and $F$ is applied to $I$ using discrete convolution, i.e., $C_{i,j} = \sum_{k,l} F_{k,l} I_{i-k,j-l}$. Care must be taken that indices $i - k$ and $j - l$ do not exceed the image's dimensions.

In [2]:
def convolve(I, F):
    M, N = I.shape
    m, n = F.shape
    C = np.zeros((M, N))
    for i in range(M):
        i_low, i_high = max(i - m//2, 0), min(i + m//2 + 1, M)
        k_low, k_high = max(m//2 - i, 0), min(m//2 + 1 + (M - 1 - i), m)
        for j in range(N):
            j_low, j_high = max(j - n//2, 0), min(j + n//2 + 1, N)
            l_low, l_high = max(n//2 - j, 0), min(n//2 + 1 + (N - 1 - j), n)
            C[i, j] += np.sum(F[k_low:k_high, l_low:l_high]*I[i_low:i_high, j_low:j_high])
    return C

In [3]:
I = np.zeros((7, 7))
I[0, 0] = 1.0
F = np.ones((3, 3))/9.0

In [4]:
convolve(I, F)

array([[ 0.11111111,  0.11111111,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.11111111,  0.11111111,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ]])

The scipy package actually has a function to do convolution, so let's check.

In [5]:
import scipy.ndimage as sim

In [6]:
sim.convolve(I, F, mode='constant')

array([[ 0.11111111,  0.11111111,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.11111111,  0.11111111,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ]])

Note that we have to use a non-default mode, since this is the one our convolution function assumes, i.e., at the borders of the image, it is supposed that the non-existing neighbouring points are zero.  Other choices are possible.

Let's do a more thorough test.

In [7]:
I = np.random.uniform(0.0, 1.0, (100, 100))
F = np.ones((7, 7))/49.0

In [8]:
C1 = convolve(I, F)
C2 = sim.convolve(I, F, mode='constant')

Compare the results up to relative precision of $10^{-10}$.

In [9]:
np.allclose(C1, C2, rtol=1.0e-10)

True

Looks good, time both implementations.

In [10]:
%timeit convolve(I, F)

1 loop, best of 3: 147 ms per loop


In [11]:
%timeit sim.convolve(I, F, mode='constant')

The slowest run took 14.84 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 795 µs per loop


Not so good for our own implementation.

## Second approach

Instead of fiddling with indices, it would be simpler to embed the image in a larger one.

In [12]:
def convolve2(I, F):
    M, N = I.shape
    m, n = F.shape
    I_enlarge = np.zeros((M + 2*m//2 - 1, N + 2*n//2 - 1))
    I_enlarge[m//2:m//2 + M,n//2:n//2 + N] = I
    C = np.zeros((M, N))
    for i in range(m//2, m//2 + M):
        i_low, i_high = i - m//2, i + m//2 + 1
        for j in range(n//2, n//2 + N):
            j_low, j_high = j - n//2, j + n//2 + 1
            C[i - m//2, j - n//2] += np.sum(F*I_enlarge[i_low:i_high, j_low:j_high])
    return C

Compute the same example as for the first implementation.

In [13]:
I = np.zeros((7, 7))
I[0, 0] = 1.0
I[-1, -1] = 1.0
F = np.ones((3, 3))/9.0

In [14]:
C = convolve2(I, F)

In [15]:
print(C)

[[ 0.11111111  0.11111111  0.          0.          0.          0.          0.        ]
 [ 0.11111111  0.11111111  0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.11111111
   0.11111111]
 [ 0.          0.          0.          0.          0.          0.11111111
   0.11111111]]


Verify against the implementation in `scipy.ndimage`.

In [16]:
I = np.random.uniform(0.0, 1.0, (100, 100))
F = np.ones((7, 7))/49.0

In [17]:
C1 = convolve2(I, F)
C2 = sim.convolve(I, F, mode='constant')

In [18]:
np.allclose(C1, C2, rtol=1.0e-10)

True

Time the function.

In [19]:
%timeit convolve2(I, F)

1 loop, best of 3: 135 ms per loop


In [20]:
%timeit sim.convolve(I, F, mode='constant')

The slowest run took 12.78 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 815 µs per loop


Not too much of an improvement, but at least the code is somewhat simpler.